# Chat with Multiple Providers

This notebook demonstrates how to use lionpride with different LLM providers. The API is **identical** across providers - you only change the `provider` and `model` parameters.

**Key Benefits:**
- Same code works with any provider
- Easy provider switching
- Fallback chains for resilience
- Provider-specific optimizations

## Setup

First, set your API keys as environment variables (only for providers you want to use):

```bash
export OPENAI_API_KEY="sk-..."
export ANTHROPIC_API_KEY="sk-ant-..."
export GEMINI_API_KEY="..."
export GROQ_API_KEY="gsk_..."
```

In [ ]:
import os

from lionpride import Session
from lionpride.operations import communicate
from lionpride.services import iModel

# Verify at least one API key is set
providers_available = {
    "openai": bool(os.getenv("OPENAI_API_KEY")),
    "anthropic": bool(os.getenv("ANTHROPIC_API_KEY")),
    "gemini": bool(os.getenv("GEMINI_API_KEY")),
    "groq": bool(os.getenv("GROQ_API_KEY")),
}

print("Available providers:")
for provider, available in providers_available.items():
    status = "✓" if available else "✗"
    print(f"  {status} {provider}")

## Example 1: OpenAI GPT-4o

OpenAI provides the most widely-used models including GPT-4o and GPT-4o-mini.

In [ ]:
async def chat_openai():
    """Chat with OpenAI GPT-4o"""
    session = Session()

    # OpenAI configuration
    model = iModel(
        provider="openai",
        model="gpt-4o-mini",  # or "gpt-4o" for more capable model
        temperature=0.7,
        max_tokens=500,
    )
    session.services.register(model)

    branch = session.create_branch(name="openai-chat")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Explain quantum computing in simple terms",
            "imodel": model.name,
        },
    )

    print(f"OpenAI Response:\n{result}\n")
    return result


# Run if OpenAI is available
if providers_available["openai"]:
    await chat_openai()
else:
    print("⚠️ OpenAI not configured. Set OPENAI_API_KEY to run this example.")

## Example 2: Anthropic Claude

Anthropic's Claude models excel at long-context analysis and nuanced reasoning.

**Important:** Anthropic requires the `endpoint="messages"` parameter.

In [ ]:
async def chat_anthropic():
    """Chat with Anthropic Claude"""
    session = Session()

    # Anthropic configuration - note the 'messages' endpoint
    model = iModel(
        provider="anthropic",
        endpoint="messages",  # Important: Anthropic uses 'messages' endpoint
        model="claude-3-5-haiku-20241022",  # or "claude-3-5-sonnet-20241022"
        temperature=0.7,
        max_tokens=500,
    )
    session.services.register(model)

    branch = session.create_branch(name="claude-chat")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Explain quantum computing in simple terms",
            "imodel": model.name,
        },
    )

    print(f"Claude Response:\n{result}\n")
    return result


# Run if Anthropic is available
if providers_available["anthropic"]:
    await chat_anthropic()
else:
    print("⚠️ Anthropic not configured. Set ANTHROPIC_API_KEY to run this example.")

## Example 3: Google Gemini

Google's Gemini models offer strong multimodal capabilities and a generous free tier.

In [ ]:
async def chat_gemini():
    """Chat with Google Gemini"""
    session = Session()

    # Gemini configuration
    model = iModel(
        provider="gemini",
        model="gemini-2.0-flash-exp",  # or "gemini-1.5-pro" for larger model
        temperature=0.7,
        max_tokens=500,
    )
    session.services.register(model)

    branch = session.create_branch(name="gemini-chat")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Explain quantum computing in simple terms",
            "imodel": model.name,
        },
    )

    print(f"Gemini Response:\n{result}\n")
    return result


# Run if Gemini is available
if providers_available["gemini"]:
    await chat_gemini()
else:
    print("⚠️ Gemini not configured. Set GEMINI_API_KEY to run this example.")

## Example 4: Multi-Provider Comparison

Ask the same question to multiple providers and compare responses.

In [ ]:
async def compare_providers():
    """Compare responses from different providers"""
    session = Session()

    # Register multiple models
    model_configs = []

    if providers_available["openai"]:
        model_configs.append(iModel(provider="openai", model="gpt-4o-mini", temperature=0.7))

    if providers_available["anthropic"]:
        model_configs.append(
            iModel(
                provider="anthropic",
                endpoint="messages",
                model="claude-3-5-haiku-20241022",
                temperature=0.7,
            )
        )

    if providers_available["gemini"]:
        model_configs.append(
            iModel(provider="gemini", model="gemini-2.0-flash-exp", temperature=0.7)
        )

    if not model_configs:
        print("⚠️ No providers configured. Set at least one API key.")
        return

    for model in model_configs:
        session.services.register(model)

    question = "What is the difference between AI and machine learning?"

    print(f"Question: {question}\n")

    # Ask same question to all models
    for model in model_configs:
        branch = session.create_branch(name=f"{model.name}-branch")

        result = await communicate(
            session=session,
            branch=branch,
            parameters={
                "instruction": question,
                "imodel": model.name,
            },
        )

        print(f"\n{'=' * 60}")
        print(f"Provider: {model.backend.config.provider}")
        print(f"Model: {model.backend.config.model}")
        print(f"{'=' * 60}")
        print(result)
        print()


await compare_providers()

## Example 5: Dynamic Provider Switching

Switch providers based on requirements (cost, speed, capabilities).

In [ ]:
async def smart_chat(question: str, priority: str = "balanced"):
    """
    Choose provider based on priority:
    - 'speed': Groq (fastest)
    - 'cost': Gemini (most affordable)
    - 'quality': OpenAI GPT-4o (highest quality)
    - 'balanced': Claude Haiku (good balance)
    """
    session = Session()

    # Provider selection strategy
    strategies = {
        "speed": ("groq", "llama-3.1-70b-versatile", {}),
        "cost": ("gemini", "gemini-2.0-flash-exp", {}),
        "quality": ("openai", "gpt-4o", {}),
        "balanced": ("anthropic", "claude-3-5-haiku-20241022", {"endpoint": "messages"}),
    }

    provider, model_name, kwargs = strategies.get(priority, strategies["balanced"])

    # Check if provider is available
    if not providers_available.get(provider, False):
        print(f"⚠️ {provider} not configured. Trying fallback...")
        # Try first available provider
        for p, available in providers_available.items():
            if available:
                provider = p
                if provider == "openai":
                    model_name = "gpt-4o-mini"
                    kwargs = {}
                elif provider == "anthropic":
                    model_name = "claude-3-5-haiku-20241022"
                    kwargs = {"endpoint": "messages"}
                elif provider == "gemini":
                    model_name = "gemini-2.0-flash-exp"
                    kwargs = {}
                elif provider == "groq":
                    model_name = "llama-3.1-70b-versatile"
                    kwargs = {}
                break
        else:
            raise RuntimeError("No providers configured")

    model = iModel(provider=provider, model=model_name, temperature=0.7, **kwargs)
    session.services.register(model)

    branch = session.create_branch(name="smart-chat")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": question,
            "imodel": model.name,
        },
    )

    print(f"Priority: {priority}")
    print(f"Selected: {provider} / {model_name}")
    print(f"\nResponse:\n{result}\n")
    return result


# Try different priorities
question = "Explain REST APIs in one sentence."

for priority in ["speed", "cost", "balanced"]:
    try:
        await smart_chat(question, priority=priority)
    except Exception as e:
        print(f"Failed with {priority}: {e}\n")

## Example 6: Custom Endpoints

Use custom API endpoints for providers like Groq, OpenRouter, or self-hosted models.

In [ ]:
async def chat_groq():
    """Chat with Groq - extremely fast inference"""
    if not providers_available.get("groq", False):
        print("⚠️ Groq not configured. Set GROQ_API_KEY to run this example.")
        return

    session = Session()

    # Groq configuration - blazing fast
    model = iModel(
        provider="groq",
        model="llama-3.1-70b-versatile",  # or "mixtral-8x7b-32768"
        temperature=0.7,
        max_tokens=500,
    )
    session.services.register(model)

    branch = session.create_branch(name="groq-chat")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Explain quantum computing in simple terms",
            "imodel": model.name,
        },
    )

    print(f"Groq Response (fast!):\n{result}\n")
    return result


await chat_groq()

### Custom Base URL (Self-hosted or Proxy)

You can point to custom API endpoints for self-hosted models or API proxies.

In [ ]:
# Example: Custom OpenAI-compatible endpoint
# (Don't run this unless you have a custom endpoint)


async def chat_custom_endpoint():
    """Example with custom API endpoint (not runnable without actual endpoint)"""
    session = Session()

    model = iModel(
        provider="openai",  # OpenAI-compatible API
        model="custom-model",
        api_key="your-custom-key",
        base_url="https://your-api-proxy.com/v1",  # Custom endpoint
        temperature=0.7,
    )
    session.services.register(model)

    branch = session.create_branch(name="custom-chat")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Hello from custom endpoint!",
            "imodel": model.name,
        },
    )

    return result


print("Custom endpoint example (code only - requires actual endpoint)")
# await chat_custom_endpoint()  # Uncomment if you have a custom endpoint

## Example 7: Provider Resilience with Fallback Chain

Implement automatic fallback to handle provider outages or rate limits.

In [ ]:
async def chat_with_fallback(question: str):
    """Try multiple providers until one succeeds"""
    session = Session()

    # Define fallback chain (order by preference)
    providers = [
        ("openai", "gpt-4o-mini", {}),
        ("anthropic", "claude-3-5-haiku-20241022", {"endpoint": "messages"}),
        ("gemini", "gemini-2.0-flash-exp", {}),
        ("groq", "llama-3.1-70b-versatile", {}),
    ]

    for provider, model_name, kwargs in providers:
        # Skip if provider not configured
        if not providers_available.get(provider, False):
            print(f"⊘ {provider} not configured, skipping...")
            continue

        try:
            model = iModel(provider=provider, model=model_name, temperature=0.7, **kwargs)
            session.services.register(model)
            branch = session.create_branch(name=f"{provider}-branch")

            result = await communicate(
                session=session,
                branch=branch,
                parameters={
                    "instruction": question,
                    "imodel": model.name,
                },
            )

            print(f"✓ Success with {provider}")
            print(f"\nResponse:\n{result}\n")
            return result

        except Exception as e:
            print(f"✗ {provider} failed: {e}")
            continue

    raise RuntimeError("All providers failed")


# Test fallback chain
await chat_with_fallback("What is recursion?")

## Provider Comparison Reference

| Provider | Endpoint | Best For | Speed | Cost |
|----------|----------|----------|-------|------|
| OpenAI | `chat/completions` (default) | General purpose, reasoning | Medium | Medium |
| Anthropic | `messages` (required) | Long context, analysis | Medium | Medium |
| Gemini | Default | Multimodal, free tier | Fast | Low |
| Groq | Default | Speed-critical apps | Very Fast | Low |

### Common Pitfalls

1. **Wrong endpoint for Anthropic** - Always use `endpoint="messages"`
2. **Missing API keys** - Set environment variables before running
3. **Provider-specific model names** - Each provider has different model identifiers
4. **Rate limits** - Implement retry logic or fallback chains for production

## Summary

**Key Takeaways:**

1. **Unified API**: Same code works with all providers
2. **Easy Switching**: Change `provider` and `model` parameters only
3. **Fallback Chains**: Build resilient systems with automatic provider failover
4. **Flexible Configuration**: Support for custom endpoints and parameters
5. **Provider-Specific Optimizations**: Each provider has unique strengths

**Next Steps:**
- Learn about [structured outputs](03_structured_outputs.ipynb)
- Explore [multi-turn conversations](04_multi_turn_chat.ipynb)
- Try [tool calling](05_tool_calling.ipynb)

**Resources:**
- [Chat Cookbook](../docs/cookbook/chat.md)
- [API Reference: iModel](../docs/api/services.md)
- [API Reference: Providers](../docs/api/providers.md)